In [1]:
%matplotlib inline

import glob
import pandas as pd
import yaml
import numpy as np
import cv2
import time
from skimage import exposure
from skimage import feature
import matplotlib.pyplot as plt

In [2]:
prefix = 'C:\\Users\\rafae\\Desktop\\Coleta\\faces\\training_*.csv'

In [3]:
import yaml
skip_lines = 3
def read_yaml_depth(file):
    file_yaml = file.replace('bmp', 'yaml')
    print(file_yaml)
    with open(file_yaml) as infile:
        for i in range(skip_lines):
            _ = infile.readline()
        data = yaml.load(infile)
    return np.reshape(data['data'], (data['rows'], data['cols']))

In [4]:
def show_hog_img(img, hog_img):
    for i in range(4):
        hog_img[i] = exposure.rescale_intensity(hog_img[i], out_range=(0, 255))
        hog_img[i] = hog_img[i].astype("uint8")

    img_concat = np.concatenate((img[0], img[1],img[2],img[3]), axis = 1)
    img_hog_concat = np.concatenate((img[0], hog_img[1],hog_img[2],hog_img[3]), axis = 1)
    
    img_total = np.concatenate((img_concat, img_hog_concat), axis = 0)
    
    fig, ax = plt.subplots(nrows=1, ncols=4)
    for i in range(4):
        plt.subplot(1, 4, i+1)
        plt.hist(hog_img[i])
    plt.show()

    cv2.imshow("HOG Image", img_total)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
show_hog_image = False

def calculate_hog(row):
    tic = time.clock()
    
    # improvement agrupar
    #dep = cv2.imread(row['depfile_path'],0)
    dep = read_yaml_depth(row['depfile_path'])
    dep = dep[row['depxs']:row['depxe'],row['depys']:row['depye']]
    dep = (dep - dep.mean()) / dep.std()
    (H_dep, dep_hog) = feature.hog(dep, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    print(dep.shape, dep_hog.shape, H_dep.shape)
    
    ir1 = cv2.imread(row['ir1file_path'],0)
    ir1 = ir1[row['ir1xs']:row['ir1xe'],row['ir1ys']:row['ir1ye']]
    (H_ir1, ir1_hog) = feature.hog(ir1, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #print(ir1.shape, ir1_hog.shape, H_ir1.shape)
    
    ir2 = cv2.imread(row['ir2file_path'],0)
    ir2 = ir2[row['ir2xs']:row['ir2xe'],row['ir2ys']:row['ir2ye']]
    (H_ir2, ir2_hog) = feature.hog(ir2, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #print(ir2.shape, ir2_hog.shape, H_ir2.shape)
    
    rgb = cv2.imread(row['rgbfile_path'],0)
    rgb = rgb[row['rgbxs']:row['rgbxe'],row['rgbys']:row['rgbye']]
    (H_rgb, rgb_hog) = feature.hog(rgb, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #print(rgb.shape, rgb_hog.shape, H_rgb.shape)
    
    data = [row['class'], row['person']]
    data.extend(H_dep)
    data.extend(H_ir1)
    data.extend(H_ir2)
    data.extend(H_rgb)
    
    data_hog.append(data)
    
    if show_hog_image == True:
        show_hog_img([dep,ir1,ir2,rgb], [dep_hog, ir1_hog, ir2_hog, rgb_hog])
        
    print(ds.shape, len(data_hog), row['depfile_path'], (time.clock() - tic))

In [6]:
# import the necessary packages
from skimage import feature
import numpy as np
 
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        # store the number of points and radius
        self.numPoints = numPoints
        self.radius = radius
 
    def describe(self, image, eps=1e-7):
        # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        lbp = feature.local_binary_pattern(image, self.numPoints,
            self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(),
            bins=np.arange(0, self.numPoints + 3),
            range=(0, self.numPoints + 2))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        # return the histogram of Local Binary Patterns
        return (hist, lbp)

In [7]:
show_lbp_image = False

def calculate_lbp(row):
    tic = time.clock()
    desc = LocalBinaryPatterns(24, 3)
    
    # improvement agrupar
    depr = read_yaml_depth(row['depfile_path'])
    depr = depr[row['depxs']:row['depxe'],row['depys']:row['depye']]
    depr = (depr - depr.mean()) / depr.std()
    #(H_dep, dep_hog) = feature.hog(dep, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    (H_depr, depr_hog) = desc.describe(depr)
    print(H_depr.shape)
    #dep = (dep - dep.mean()) / dep.std()
    #dep = cv2.imread(row['depfile_path'],0)    
    #dep = dep[row['depxs']:row['depxe'],row['depys']:row['depye']]
    
    #(H_dep, dep_hog) = feature.hog(dep, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2,2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    #(H_dep, dep_hog) = desc.describe(dep)
    #print(H_dep.shape)
    
    ir1 = cv2.imread(row['ir1file_path'],0)
    ir1 = ir1[row['ir1xs']:row['ir1xe'],row['ir1ys']:row['ir1ye']]
    #(H_ir1, ir1_hog) = feature.hog(ir1, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    (H_ir1, ir1_hog) = desc.describe(ir1)
    print(H_ir1.shape)
    #print(ir1.shape, ir1_hog.shape, H_ir1.shape)
    
    ir2 = cv2.imread(row['ir2file_path'],0)
    ir2 = ir2[row['ir2xs']:row['ir2xe'],row['ir2ys']:row['ir2ye']]
    #(H_ir2, ir2_hog) = feature.hog(ir2, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    (H_ir2, ir2_hog) = desc.describe(ir2)
    print(H_ir2.shape)
    #print(ir2.shape, ir2_hog.shape, H_ir2.shape)
    
    rgb = cv2.imread(row['rgbfile_path'],0)
    rgb = rgb[row['rgbxs']:row['rgbxe'],row['rgbys']:row['rgbye']]
    #(H_rgb, rgb_hog) = feature.hog(rgb, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2), transform_sqrt=True, visualise=True, block_norm='L2-Hys')
    (H_rgb, rgb_hog) = desc.describe(rgb)
    print(H_rgb.shape)
    #print(rgb.shape, rgb_hog.shape, H_rgb.shape)
    
    data = [row['class'], row['person']]
    data.extend(H_depr)
    #data.extend(H_dep)
    data.extend(H_ir1)
    data.extend(H_ir2)
    data.extend(H_rgb)
    
    data_hog.append(data)
    
    if show_lbp_image == True:
        show_hog_img([dep,ir1,ir2,rgb], [dep_hog, ir1_hog, ir2_hog, rgb_hog])
        
    print(ds.shape, len(data_hog), row['depfile_path'], (time.clock() - tic))

In [14]:
files_list = glob.glob(prefix)
print(len(files_list))

518


In [15]:
files_list = files_list[235:]
print(len(files_list))

283


In [16]:
feature_desc = 'hog'

for f in files_list:
    #f = files_list[0]
    ds = pd.read_csv(f, header=0)
    print(ds.shape, f)

    data_hog = []
    tic = time.clock()
    savefile = False
    for index, row in ds.iterrows():
        if feature_desc == 'lbp':
            calculate_lbp(row)
        else:
            calculate_hog(row)
    print(time.clock() - tic)

    if len(data_hog) > 0:
        print(len(data_hog[0]))
        # save file
        df = pd.DataFrame(data_hog)
        filename = 'features_{:s}_{:s}_{:d}.csv'.format(feature_desc, ds['person'][0],ds['class'][0])
        df.to_csv(filename, index=False, encoding='utf-8')

(15, 22) C:\Users\rafae\Desktop\Coleta\faces\training_p019_1.csv
E:\MestradoData\p019\19_1_depth_1.yaml


C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:144: RuntimeWarning: invalid value encountered in sqrt
  image = np.sqrt(image)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:190: RuntimeWarning: invalid value encountered in remainder
  orientations, orientation_histogram)
C:\ProgramData\Anaconda3\lib\site-packages\skimage\feature\_hog.py:17: RuntimeWarning: invalid value encountered in minimum
  out = np.minimum(out, 0.2)


(104, 88) (104, 88) (4320,)
(15, 22) 1 E:\MestradoData\p019\19_1_depth_1.bmp 6.546490827464037
E:\MestradoData\p019\19_1_depth_11.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 2 E:\MestradoData\p019\19_1_depth_11.bmp 6.26671860407419
E:\MestradoData\p019\19_1_depth_21.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 3 E:\MestradoData\p019\19_1_depth_21.bmp 6.507392389220829
E:\MestradoData\p019\19_1_depth_31.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 4 E:\MestradoData\p019\19_1_depth_31.bmp 6.46359509049277
E:\MestradoData\p019\19_1_depth_41.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 5 E:\MestradoData\p019\19_1_depth_41.bmp 6.4014076828383715
E:\MestradoData\p019\19_1_depth_51.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 6 E:\MestradoData\p019\19_1_depth_51.bmp 5.999046321741183
E:\MestradoData\p019\19_1_depth_61.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 7 E:\MestradoData\p019\19_1_depth_61.bmp 5.901374497679413
E:\MestradoData\p019\19_1_depth_71.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 8 E:\Mestrado

(15, 22) 13 E:\MestradoData\p019\19_12_depth_151.bmp 6.136126707257915
E:\MestradoData\p019\19_12_depth_161.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 14 E:\MestradoData\p019\19_12_depth_161.bmp 6.204439147109781
E:\MestradoData\p019\19_12_depth_171.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 15 E:\MestradoData\p019\19_12_depth_171.bmp 6.267197813572068
93.83622373434179
17282
(15, 22) C:\Users\rafae\Desktop\Coleta\faces\training_p019_13.csv
E:\MestradoData\p019\19_13_depth_21.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 1 E:\MestradoData\p019\19_13_depth_21.bmp 6.847625601876075
E:\MestradoData\p019\19_13_depth_31.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 2 E:\MestradoData\p019\19_13_depth_31.bmp 6.111754578366742
E:\MestradoData\p019\19_13_depth_41.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 3 E:\MestradoData\p019\19_13_depth_41.bmp 6.169028212125909
E:\MestradoData\p019\19_13_depth_51.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 4 E:\MestradoData\p019\19_13_depth_51.bmp 6.454814060393801
E:\Me

(104, 88) (104, 88) (4320,)
(15, 22) 10 E:\MestradoData\p019\19_16_depth_91.bmp 6.014240778871454
E:\MestradoData\p019\19_16_depth_101.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 11 E:\MestradoData\p019\19_16_depth_101.bmp 5.973929107018989
E:\MestradoData\p019\19_16_depth_111.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 12 E:\MestradoData\p019\19_16_depth_111.bmp 6.093512222410141
E:\MestradoData\p019\19_16_depth_121.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 13 E:\MestradoData\p019\19_16_depth_121.bmp 6.239348749156534
E:\MestradoData\p019\19_16_depth_131.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 14 E:\MestradoData\p019\19_16_depth_131.bmp 5.800107614729768
E:\MestradoData\p019\19_16_depth_141.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 15 E:\MestradoData\p019\19_16_depth_141.bmp 6.207657712467949
95.59429878968979
17282
(15, 22) C:\Users\rafae\Desktop\Coleta\faces\training_p019_17.csv
E:\MestradoData\p019\19_17_depth_11.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 1 E:\MestradoData\p019\19_17_

...

(15, 22) 9 E:\MestradoData\p032\32_9_depth_101.bmp 5.839686447656277
E:\MestradoData\p032\32_9_depth_111.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 10 E:\MestradoData\p032\32_9_depth_111.bmp 5.972086540572491
E:\MestradoData\p032\32_9_depth_121.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 11 E:\MestradoData\p032\32_9_depth_121.bmp 6.10406470790258
E:\MestradoData\p032\32_9_depth_131.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 12 E:\MestradoData\p032\32_9_depth_131.bmp 5.913292315424769
E:\MestradoData\p032\32_9_depth_141.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 13 E:\MestradoData\p032\32_9_depth_141.bmp 5.887107249694964
E:\MestradoData\p032\32_9_depth_151.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 14 E:\MestradoData\p032\32_9_depth_151.bmp 5.877457874601532
E:\MestradoData\p032\32_9_depth_161.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 15 E:\MestradoData\p032\32_9_depth_161.bmp 5.777092521802842
89.7606713989735
17282
(15, 22) C:\Users\rafae\Desktop\Coleta\faces\training_p033_1.csv
E:\Mestrad

(15, 22) 6 E:\MestradoData\p033\33_12_depth_51.bmp 5.923461196278367
E:\MestradoData\p033\33_12_depth_61.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 7 E:\MestradoData\p033\33_12_depth_61.bmp 6.071543647176441
E:\MestradoData\p033\33_12_depth_71.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 8 E:\MestradoData\p033\33_12_depth_71.bmp 5.8275422597216675
E:\MestradoData\p033\33_12_depth_81.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 9 E:\MestradoData\p033\33_12_depth_81.bmp 5.881933130323887
E:\MestradoData\p033\33_12_depth_91.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 10 E:\MestradoData\p033\33_12_depth_91.bmp 6.0869067955318314
E:\MestradoData\p033\33_12_depth_101.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 11 E:\MestradoData\p033\33_12_depth_101.bmp 5.84306066721183
E:\MestradoData\p033\33_12_depth_111.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 12 E:\MestradoData\p033\33_12_depth_111.bmp 6.130020638011047
E:\MestradoData\p033\33_12_depth_121.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 13 E:\MestradoDa

(104, 88) (104, 88) (4320,)
(15, 22) 2 E:\MestradoData\p033\33_17_depth_11.bmp 6.01583406156351
E:\MestradoData\p033\33_17_depth_21.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 3 E:\MestradoData\p033\33_17_depth_21.bmp 5.81895125416122
E:\MestradoData\p033\33_17_depth_31.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 4 E:\MestradoData\p033\33_17_depth_31.bmp 5.985881690168753
E:\MestradoData\p033\33_17_depth_41.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 5 E:\MestradoData\p033\33_17_depth_41.bmp 5.90802456649908
E:\MestradoData\p033\33_17_depth_51.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 6 E:\MestradoData\p033\33_17_depth_51.bmp 5.8744968892860925
E:\MestradoData\p033\33_17_depth_61.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 7 E:\MestradoData\p033\33_17_depth_61.bmp 5.785354441202799
E:\MestradoData\p033\33_17_depth_71.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 8 E:\MestradoData\p033\33_17_depth_71.bmp 5.874770271788293
E:\MestradoData\p033\33_17_depth_101.yaml
(104, 88) (104, 88) (4320,)
(15, 22

(15, 22) 14 E:\MestradoData\p033\33_2_depth_141.bmp 5.9025332128148875
E:\MestradoData\p033\33_2_depth_151.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 15 E:\MestradoData\p033\33_2_depth_151.bmp 6.037970932964527
89.08060912149449
17282
(15, 22) C:\Users\rafae\Desktop\Coleta\faces\training_p033_3.csv
E:\MestradoData\p033\33_3_depth_1.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 1 E:\MestradoData\p033\33_3_depth_1.bmp 6.6289480462619395
E:\MestradoData\p033\33_3_depth_11.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 2 E:\MestradoData\p033\33_3_depth_11.bmp 5.80126000888049
E:\MestradoData\p033\33_3_depth_21.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 3 E:\MestradoData\p033\33_3_depth_21.bmp 5.849533748514659
E:\MestradoData\p033\33_3_depth_31.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 4 E:\MestradoData\p033\33_3_depth_31.bmp 5.83287558883967
E:\MestradoData\p033\33_3_depth_41.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 5 E:\MestradoData\p033\33_3_depth_41.bmp 5.910839379088429
E:\MestradoData\p033\33_

(104, 88) (104, 88) (4320,)
(15, 22) 12 E:\MestradoData\p033\33_6_depth_131.bmp 5.851470339577645
E:\MestradoData\p033\33_6_depth_141.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 13 E:\MestradoData\p033\33_6_depth_141.bmp 5.880265181025607
E:\MestradoData\p033\33_6_depth_151.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 14 E:\MestradoData\p033\33_6_depth_151.bmp 5.990169686778245
E:\MestradoData\p033\33_6_depth_161.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 15 E:\MestradoData\p033\33_6_depth_161.bmp 5.884021424968523
89.63213263633952
17282
(15, 22) C:\Users\rafae\Desktop\Coleta\faces\training_p033_7.csv
E:\MestradoData\p033\33_7_depth_31.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 1 E:\MestradoData\p033\33_7_depth_31.bmp 6.816294466138061
E:\MestradoData\p033\33_7_depth_41.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 2 E:\MestradoData\p033\33_7_depth_41.bmp 6.133648091930809
E:\MestradoData\p033\33_7_depth_51.yaml
(104, 88) (104, 88) (4320,)
(15, 22) 3 E:\MestradoData\p033\33_7_depth_51.bmp 6.1109

In [ ]:
from sklearn.metrics import make_scorer,precision_recall_fscore_support


make_scorer(precision_recall_fscore_support, labels = [1,2,3])

In [ ]:
ds.head()